<a href="https://colab.research.google.com/github/tanzmorshed/tra301-final/blob/main/Keyword_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install num2words
!pip install --upgrade gensim
!pip install fuzzywuzzy
!pip install syllables

     |████████████████████████████████| 101 kB 5.1 MB/s 
     |████████████████████████████████| 24.1 MB 67.7 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
from gensim.parsing.preprocessing import remove_stopwords
import re
import csv
import math
import num2words
from fuzzywuzzy import fuzz
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
import syllables
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('sentiwordnet')
nltk.download('wordnet')
# Import NLTK and stopwords
import nltk
from nltk import tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

import glob
import os
import re

from textblob import TextBlob

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
!git clone https://github.com/tanzmorshed/tra301-final.git

Cloning into 'tra301-final'...
remote: Enumerating objects: 2298, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2298 (delta 4), reused 14 (delta 1), pack-reused 2280
Receiving objects: 100% (2298/2298), 526.05 MiB | 12.81 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Checking out files: 100% (2859/2859), done.


In [4]:
# Get movie scripts from each genre and put them into corresponding lists

genre_strings = [  'Action', 'Adventure', 'Animation', 'Biography', 'Comedy',
            'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
            'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
            'Short', 'Sport', 'Thriller', 'War', 'Western']

action, adventure, animation, biography, comedy, crime, drama, family, fantasy, noir, history, horror, music, musical, mystery, romance, scifi, short, sport, thriller, war, western = [[] for _ in range(22)]

genres = [ action, adventure, animation, biography, comedy, crime, drama,
          family, fantasy, noir, history, horror, music, musical, mystery,
          romance, scifi, short, sport, thriller, war, western]

for i, genre in enumerate(genres):
    path = f'/content/tra301-final/imsdb_raw_nov_2015/{genre_strings[i]}'
    for filename in glob.glob(os.path.join(path, '*.txt')):
        # print(filename)
        with open(filename, 'r') as f: # open in readonly mode
            genre.append((os.path.basename(filename), f.read()))

In [10]:
sum = 0
movies = []
for genre in genres:
  movies += genre
print(len((movies)))

2831


In [ ]:
# Reads train.csv to create genre buckets of keywords

genre_categories = []
genre_keywords_lists = []

path = f'/content/tra301-final/train.csv'
with open(path, 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader) #skips header
    for row in csv_reader:
        word = row[0]
        genre = row[1]
        if genre not in genre_categories:
          genre_categories.append(genre)
          genre_index = len(genre_categories)-1
          genre_keywords_lists.append([])
        else:
          genre_index = genre_categories.index(genre)
        if len(word.split()) == 1:
          genre_keywords_lists[genre_index].append(word)

genre_keywords = []
for item in genre_keywords_lists:
  genre_keywords.append(set(item))


In [ ]:
# Given a script and the list of keywords for a given genre, returns the percentage
# of words in the script that belong to the genre's keywords
def genre_score(script, genre_keyword_list):
  script_words = script.split()
  if len(script_words) == 0:
    return 0

  sum = 0
  for word in script_words:
    if word in genre_keyword_list:
      sum += 1
  return sum / len(script_words)

In [ ]:
# Clusters data based on genre scores across the 10 genres identified in the
# word dataset


import numpy as np
from sklearn import preprocessing

def get_all_genre_scores(script):
  scores = []
  for genre in genre_keywords:
    scores.append(genre_score(script, genre))
  return scores

cluster1 = {'Short', 'Drama', 'Comedy', 'Romance', 'Family', 'Music', 'Fantasy', 'Sport', 'Musical'}
cluster2 = {'Thriller', 'Horror', 'Action', 'Crime', 'Adventure', 'Sci-Fi', 'Mystery', 'Animation', 'Western'}
cluster3 = {'Documentary', 'History', 'Biography', 'War', 'News'}
 

genre_vectors = []
movie_names = []


movie_info = []
for i, genre in enumerate(genres):
  gen_name = genre_strings[i]
  cluster_label = None
  if gen_name in cluster1:
    cluster_label = 'A'
  elif gen_name in cluster2:
    cluster_label = 'B'
  elif gen_name in cluster3:
    cluster_label = 'C'
  else:
    cluster_label = 'D' # Should just be Film-Noir
  for script in genre:
    movie_info.append((script[0], gen_name, cluster_label))
    genre_vectors.append(get_all_genre_scores(script[1]))
    movie_names.append(script[0])


a = np.vstack(genre_vectors)
a = preprocessing.normalize(a)
with open('genrevectors.txt', 'w') as f:
  for v, vec in enumerate(a):
    f.write(movie_names[v] + '\n')
    f.write(str(vec.tolist()) + '\n')
  f.close()
print(a)

[[0.52510384 0.41746431 0.28431342 ... 0.14495374 0.32132027 0.09645141]
 [0.51635718 0.42017841 0.26093838 ... 0.11639425 0.32600049 0.11094366]
 [0.53800521 0.41576786 0.24554697 ... 0.10843204 0.29956195 0.10139535]
 ...
 [0.50492089 0.42421843 0.31039902 ... 0.14044157 0.30654074 0.0989007 ]
 [0.53725577 0.43465389 0.2850661  ... 0.12369759 0.30232623 0.09643755]
 [0.49286677 0.41333233 0.31035516 ... 0.14910335 0.33583845 0.10876667]]


In [ ]:
print(a.shape)

(2831, 10)


In [ ]:
import sklearn.cluster as cluster

def cluster_vectors(features, num_clusters):
  kmeans = cluster.KMeans(num_clusters, n_init=50, random_state=42).fit(features)
  return kmeans.labels_

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

def visualize_clusters(features, movie_info, num_clusters):
  cluster_list = cluster_vectors(features, num_clusters)
  movie_df = pd.DataFrame(features)
  n = 2
  tsne = TSNE(n_components=n, perplexity=30, random_state=0)
  movie_embedding = tsne.fit_transform(movie_df)
  tsne_df = pd.DataFrame(data = movie_embedding, columns = list(range(n)))
  tsne_df['cluster'] = pd.DataFrame(cluster_list)
  movies = [x[0] for x in movie_info]
  genres = [x[1] for x in movie_info]
  labels = [x[2] for x in movie_info]
  tsne_df['movie'] = pd.DataFrame(movies)
  tsne_df['genre'] = pd.DataFrame(genres)
  tsne_df['label'] = pd.DataFrame(labels)

  fig = px.scatter(tsne_df, 
                  x=0,
                  y=1,
                  hover_data=['movie','genre'],
                  color='cluster',
                  opacity=.8,
                  template='simple_white')\
            .update_traces(marker=dict(size=15))\
            .update_layout(
                title={
                    'text': "Script Clusterings",
                    'y':0.95,
                    'x':0.19,
                    'xanchor': 'center',
                    'yanchor': 'top'})
  fig.show()
  tsne_df.sort_values(by=['cluster', 'genre', 'label'], inplace=True)
  display(tsne_df)
  return tsne_df

n_clusters = 22 ### amount of clusters; feel free to change this!
tsne_df = visualize_clusters(a, movie_info, n_clusters)


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


,0,1,cluster,movie,genre,label
25,2.106816,-21.492577,0,startrek.txt,Action,B
33,-1.173273,-22.089968,0,matrixthe.txt,Action,B
46,-3.381155,-42.349930,0,fairgame.txt,Action,B
49,-11.392734,-25.326050,0,watchmen.txt,Action,B
63,5.152409,-26.213482,0,startrekiithewrathofkhan.txt,Action,B
...,...,...,...,...,...,...
2312,-122.810837,25.964243,21,predator.txt,Sci-Fi,B
2353,-128.582932,25.305407,21,madmax2theroadwarrior.txt,Sci-Fi,B
2367,-132.481949,26.431992,21,walle.txt,Sci-Fi,B
2561,-121.731232,26.506409,21,predator.txt,Thriller,B


In [ ]:
# Rand index calculation
num_rows = tsne_df.shape[0]
clusters = tsne_df['cluster']
genre_by_cluster = tsne_df['genre']
titles = tsne_df['movie']

correct = 0
repeats = 0

for i in range(num_rows-1):
  i_cluster = clusters[i]
  i_genre = genre_by_cluster[i]
  i_title = titles[i]
  for j in range(i+1, num_rows):
      j_cluster = clusters[j]
      j_genre = genre_by_cluster[j]
      j_title = titles[j]
      if i_cluster == j_cluster and i_genre == j_genre:
          correct += 1
      elif i_cluster != j_cluster and i_genre != j_genre:
          correct += 1  
      if i_title == j_title:
        repeats += 1

n_choose_2 = (num_rows * (num_rows-1))/2
rand_index = (correct/(n_choose_2-repeats))
print(rand_index)

KeyboardInterrupt: ignored

In [ ]:
# Checking calculation after realizing there was a python package that does this already
import sklearn

rand= sklearn.metrics.rand_score(genre_by_cluster, clusters)

print(rand)
print(sklearn.metrics.adjusted_rand_score(genre_by_cluster, clusters)
)

In [ ]:
# comparing to tri-cluster 
tri_cluster_df = visualize_clusters(a, movie_info, 3)


num_rows = tri_cluster_df.shape[0]
tri_clusters = tri_cluster_df['cluster']
labels = tri_cluster_df['label']

print(sklearn.metrics.adjusted_rand_score(labels, tri_clusters))
